In [1]:
import collections
import csv
import os
import pyreadr
import sklearn

import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn import preprocessing
from sklearn.linear_model import LinearRegression

LEARNING_RATE = 0.005
EPOCHS = 1000
MOMENTUM = 0.75
NEURONS = 1000
BATCHES = 256
VALIDATION = 0.1
TARGET_NAME = "log2relT"

"""
STEPS
-----
DATA
1. Load indices for testing data
2. Load the complete data
3. Get the gene expression data from the complete data
4. Split the gene expression data into training and testing data
5. Get the target data from the complete data
6. Split the target data into training and testing data
7. Get the fluxomic data from the complete data
8. Split the fluxomic data into training and testing data
9. Train, evaluate, and save the model
10. Load the independent data (Linear Regression applied to generate missing data)
11. Load the model and evaluate it on the independent data
"""

'\nSTEPS\n-----\nDATA\n1. Load indices for testing data\n2. Load the complete data\n3. Get the gene expression data from the complete data\n4. Split the gene expression data into training and testing data\n5. Get the target data from the complete data\n6. Split the target data into training and testing data\n7. Get the fluxomic data from the complete data\n8. Split the fluxomic data into training and testing data\n9. Train, evaluate, and save the model\n10. Load the independent data (Linear Regression applied to generate missing data)\n11. Load the model and evaluate it on the independent data\n'

In [2]:
# TESTING_DATA_INDICES
# Indices corresponding to the testing set
DATA_DIR = "../../../data/models/moma/"
PREDICTIONS_DIR = "../../../data/predictions/moma/"

with open(DATA_DIR + "indices_for_testing_data.csv", "r") as csvfile:
    testing_data_indices = []
    for row in csv.reader(csvfile, delimiter=";"):
        testing_data_indices.append(row[0])  # careful here with [0]

testing_data_indices = list(map(int, testing_data_indices))  # fixed testing indexes

In [3]:
def check_file_read_ok(data: collections.OrderedDict, name: str) -> None:
    if data is None:
        print("error with " + name + " data not read")
    else:
        print("Success in loading " + name)
        print(data.shape)

In [4]:
# FULL_DATA
# Load the complete dataset
full_data: collections.OrderedDict = pyreadr.read_r(DATA_DIR + "complete_dataset.RDS")[None]
print(type(full_data))
check_file_read_ok(data=full_data, name="full data")
# Remove columns consisting of only zeros
full_data = full_data.loc[:, (full_data != 0).any(axis=0)]
print(type(full_data))

<class 'pandas.core.frame.DataFrame'>
Success in loading full data
(1143, 9666)
<class 'pandas.core.frame.DataFrame'>


In [5]:
# GENE_EXPRESSION_DATA
# Get the gene expression d
gene_expression_data = pyreadr.read_r(DATA_DIR + "gene_expression_dataset.RDS")[None]
check_file_read_ok(gene_expression_data, "expression data")
# Remove columns consisting of only zeros
gene_expression_data = gene_expression_data.loc[:, (gene_expression_data != 0).any(axis=0)]
# Drop target data from gene expression data
gene_expression_data = gene_expression_data.drop(columns=[TARGET_NAME])

Success in loading expression data
(1143, 6171)


In [6]:
# SPLIT GENE EXPRESSION DATA
# Split the gene expression data into training and testing
gene_expression_training_data, gene_expression_testing_data = (
    gene_expression_data.drop(gene_expression_data.index[testing_data_indices]),
    gene_expression_data.iloc[testing_data_indices, :],
)

# Scale the gene expression data
gene_expression_scaler = preprocessing.StandardScaler().fit(
    gene_expression_training_data
)
gen_expression_training_data_scaled = gene_expression_scaler.transform(
    gene_expression_training_data
).astype(np.float32)

gene_expression_testing_data_scaled = gene_expression_scaler.transform(
    gene_expression_testing_data
).astype(np.float32)


In [7]:
# TARGET_DATA
# Get the target data for evaluation
target_data = full_data[TARGET_NAME]

# Split the target data into training and testing
target_training_data, target_testing_data = (
    target_data.drop(target_data.index[testing_data_indices]),
    target_data.iloc[testing_data_indices],
)
target_testing_data.to_csv(DATA_DIR + "target_testing_data.csv")

target_training_data: pd.Series = target_training_data.astype(np.float32)
target_testing_data: pd.Series = target_testing_data.astype(np.float32)


# Drop the target data from the full data
full_data = full_data.drop(columns=[TARGET_NAME])

knockouts = full_data["Row"] # Used later

# Clean up
full_data = full_data.drop(columns="Row")

In [8]:
# FLUXOMIC_DATA
# Get fluxomic data 
fluxomic_data = full_data.drop(columns=gene_expression_data.columns.values)

In [9]:
# SPLIT FLUXOMIC DATA
# Split the target data into training and testing
fluxomic_training_data, fluxomic_testing_data = (
    fluxomic_data.drop(fluxomic_data.index[testing_data_indices]),
    fluxomic_data.iloc[testing_data_indices, :],
)

# Scale the fluxomic data
fluxome_scaler = sklearn.preprocessing.StandardScaler().fit(fluxomic_training_data)
fluxomic_training_data_scaled: np.ndarray = fluxome_scaler.transform(
    fluxomic_training_data
).astype(np.float32)
fluxomic_testing_data_scaled: np.ndarray = fluxome_scaler.transform(
    fluxomic_testing_data
).astype(np.float32)

In [10]:
def init_single_view_model(
        input_dim: int,
        model_name: str,
) -> tf.keras.Model:
    """Initialize a model with the given parameters.

    Parameters
    ----------
    input_dim : int
        The number of input features.
    model_name : str
        The name of the model. Used for naming the layers.

    Returns
    -------
    tf.keras.Model
    """
    # Input layer
    input = tf.keras.layers.Input(shape=(input_dim,))

    # Hidden layer (1)
    layer = tf.keras.layers.Dense(
        NEURONS,
        activation="sigmoid",
        kernel_constraint=tf.keras.constraints.max_norm(3),
        name=f"{model_name}_1",
    )(input)
    # Set 40% of input units to 0 at each update during training time
    layer = tf.keras.layers.Dropout(rate=0.4)(layer)

    # Hidden layer (2)
    layer = tf.keras.layers.Dense(
        NEURONS,
        activation="sigmoid",
        kernel_constraint=tf.keras.constraints.max_norm(3),
        name=f"{model_name}_2",
    )(layer)
    # Set 40% of input units to 0 at each update during training time
    layer = tf.keras.layers.Dropout(rate=0.4)(layer)

    # Final output layer
    predictions = tf.keras.layers.Dense(1, activation="linear")(layer)
    model = tf.keras.Model(inputs=input, outputs=predictions)
    print(f"Summary of the single-view model {model_name}")
    model.summary()
    return model

In [11]:
def load_single_view_model_from_file(training_data: np.ndarray, fname: str) -> tf.keras.Model:
    """Load a model from a file and return it.

    Parameters
    ----------
    data_train : np.ndarray
        The training data to load the model for.
    fname : str
        The name of the model to load.

    Returns
    -------
    tf.keras.Model
        The loaded model.
    
    """
    # Model initialization
    model = init_single_view_model(
        input_dim=training_data.shape[1],
        model_name=fname,
    )

    # Load the model weights
    #   - gene expression: expression_model.h5
    #   - fluxomic: reaction_model.h5
    model.load_weights(DATA_DIR + fname + "_weights.h5")

    return model

In [12]:
def init_multi_view_model(
    input1_dim: int,
    input2_dim: int,
    learning_rate: float,
    epochs: int,
    momentum: float,
    neurons: int,
    model_1: tf.keras.Model,
    model_2: tf.keras.Model,
) -> tf.keras.Model:
    """Initialize a model with two inputs and one output.

    Parameters
    ----------
    input1_dim : int
        The number of features in the first input.
    input2_dim : int
        The number of features in the second input.
    learning_rate : float
        The learning rate for the optimizer.
    epochs : int
        The number of epochs for training.
    momentum : float
        The momentum for the optimizer.
    neurons : int
        The number of neurons in the hidden layers.
    model_1 : tf.keras.Model
        The first model to use.
    model_2 : tf.keras.Model
        The second model to use.

    Returns
    -------
    tf.keras.Model

    """
    input_1 = tf.keras.layers.Input(shape=(input1_dim,))
    input_2 = tf.keras.layers.Input(shape=(input2_dim,))

    combined_layer = tf.keras.layers.Concatenate()([model_1(input_1), model_2(input_2)])
    combined_layer = tf.keras.layers.Dense(
        neurons,
        activation="sigmoid",
        kernel_constraint=tf.keras.constraints.max_norm(3),
        name="last_hidden",
    )(combined_layer)

    predictions = tf.keras.layers.Dense(1, activation="linear")(combined_layer)
    result = tf.keras.Model(inputs=[input_1, input_2], outputs=predictions)

    sgd = tf.keras.optimizers.SGD(
        learning_rate=learning_rate,
        weight_decay=learning_rate / epochs,
        momentum=momentum,
    )
    result.compile(
        loss="mean_squared_error", optimizer=sgd, metrics=["mean_absolute_error"]
    )
    return result

In [13]:
def train_evaluate_save_multiview_model(
    name: str,
    index: int,
    training_data_1: np.ndarray,
    training_data_2: np.ndarray,
    testing_data_1: np.ndarray,
    testing_data_2: np.ndarray,
    name_1: str,
    name_2: str,
    save_weights: bool=False,
    save_prediction: bool=True,
    verbose: bool=True,
) -> tf.keras.Model:
    """Training Multi-Modal Model on GE+MF data

    Parameters
    ----------
    name : str
        Name of the model
    index : int
        Iteration index / model index
    data_train_1 : np.ndarray
        Training data from the first modality (Metabolic Fluxes)
    data_train_2 : np.ndarray
        Training data from the second modality (Gene Expression)
    data_test_1 : np.ndarray
        Testing data from the first modality (Metabolic Fluxes)
    data_test_2 : np.ndarray
        Testing data from the second modality (Gene Expression)
    name_1 : str
        Name of the first modality
    name_2 : str
        Name of the second modality
    save_weights : bool, optional
        Save weights of the model, by default False
    save_prediction : bool, optional
        Save predictions of the model, by default True
    verbose : bool, optional
        Print verbose output, by default False

    Returns
    ----------
    tf.keras.Model
    """
    # Load the models
    print("Model 1: fluxomics")
    model_1 = load_single_view_model_from_file(
        training_data=training_data_1, fname=name_1
    )
    print("Model 2: gene expression")
    model_2 = load_single_view_model_from_file(
        training_data=training_data_2, fname=name_2
    )

    # Initialize the multi-modal model
    multi_model: tf.keras.Model = init_multi_view_model(
        input1_dim=training_data_1.shape[1],
        input2_dim=training_data_2.shape[1],
        learning_rate=LEARNING_RATE,
        epochs=EPOCHS,
        momentum=0.75,
        neurons=15,
        model_1=model_1,
        model_2=model_2,
    )
    print("multi-model summary")
    multi_model.summary()

    multi_model.compile(loss="mse", optimizer="adam", metrics=["mean_absolute_error"])

    # filename: multi_view_model_GE_MF_{index}.weights.h5
    model_file = f"{DATA_DIR}{name}_{index}.weights.h5"
    print(f"Model file: {model_file}")

    if save_weights and os.path.exists(model_file):
        multi_model.load_weights(model_file)
    else:
        multi_model.fit(
            x=[training_data_1, training_data_2],
            y=target_training_data,
            epochs=EPOCHS,
            batch_size=BATCHES,
            validation_split=VALIDATION,
            verbose=verbose,
        )

        if save_weights:
            multi_model.save_weights(model_file)
            print(f"Model weights saved to {model_file}")

    score = multi_model.evaluate(
        [testing_data_1, testing_data_2], target_testing_data, verbose=1
    )
    if save_prediction:
        predictions_file  = f"{PREDICTIONS_DIR}{name}_Predictions_{index}.csv"
        if not os.path.exists(predictions_file):
            prediction = multi_model.predict_on_batch([testing_data_1, testing_data_2])
            np.savetxt(fname=predictions_file, X=prediction, delimiter=",")
            print(f"Predictions saved to {predictions_file}")
    
    print(f"Score of the model {name}: {score}")

    return multi_model

In [14]:
model = train_evaluate_save_multiview_model(
    name="multi_view_model_GE_MF",
    index=0,
    training_data_1=fluxomic_training_data_scaled,
    training_data_2=gen_expression_training_data_scaled,
    testing_data_1=fluxomic_testing_data_scaled,
    testing_data_2=gene_expression_testing_data_scaled,
    name_1="fluxomic",
    name_2="gene_expression",
    save_weights=True,
    save_prediction=True,
    verbose=True,
)
model.save_weights(DATA_DIR + "multi_view_model_GE_MF_0.weights.h5")

Model 1: fluxomics
Summary of the single-view model fluxomic


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 1848)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ fluxomic_1 (Dense)              │ (None, 1000)           │     1,849,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1000)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ fluxomic_2 (Dense)              │ (None, 1000)           │     1,001,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 1000)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │         1,001 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,851,001 (10.88 MB)

 Trainable params: 2,851,001 (10.88 MB)

...

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2       │ (None, 1848)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_3       │ (None, 6170)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ functional_1        │ (None, 1)         │  2,851,001 │ input_layer_2[0]… │
│ (Functional)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ functional_3        │ (None, 1)         │  7,173,001 │ input_layer_3[0]… │
│ (Functional)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 2)         │          0 │ functional_1[0][… │
│ (Concatenate)       │                   │            │ functional_3[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ last_hidden (Dense) │ (None, 15)        │         45 │ concatenate[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 1)         │         16 │ last_hidden[0][0] │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 10,024,063 (38.24 MB)

 Trainable params: 10,024,063 (38.24 MB)

 Non-trainable params: 0 (0.00 B)

Model file: ../../../data/models/moma/multi_view_model_GE_MF_0.weights.h5
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0124 - mean_absolute_error: 0.0721  
Score of the model multi_view_model_GE_MF: [0.014740487560629845, 0.0749981626868248]


In [15]:
# INDEPENDENT INPUT DATASET - LINEAR REGRESSION
original_independent_data = pd.read_csv(DATA_DIR + "independent_dataset.csv")

# Preserve the 'Row' information and align columns between datasets
independent_knockouts = original_independent_data["Row"]
common_columns = original_independent_data.columns.intersection(full_data.columns)
common_independent_data = original_independent_data[common_columns]

# Identify columns in full_data not present in independent_data for prediction
missing_columns_in_independent = full_data.columns.difference(common_independent_data.columns)

# Prepare data sets for regression model
targets_impute = full_data[missing_columns_in_independent]

# Ensure the training data is ordered correctly
common_data = full_data[common_columns]
# Train the model
lm = LinearRegression()
lm.fit(
    common_data, full_data[missing_columns_in_independent]
)  # Assuming target_columns defined

# Predict
predicted_values = lm.predict(common_independent_data[common_columns])
common_independent_data[missing_columns_in_independent] = predicted_values

# Filter rows that are new knockouts (not present in the original knockouts)
new_rows = list(set(independent_knockouts) - set(knockouts))
independent_data = common_independent_data[independent_knockouts.isin(new_rows)]

/var/folders/ch/ypsvlmcx5zl0s9mxmw1v795c0000gn/T/ipykernel_18061/1363443833.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  common_independent_data[missing_columns_in_independent] = predicted_values
/var/folders/ch/ypsvlmcx5zl0s9mxmw1v795c0000gn/T/ipykernel_18061/1363443833.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  common_independent_data[missing_columns_in_independent] = predicted_values
/var/folders/ch/ypsvlmcx5zl0s9mxmw1v795c0000gn/T/ipykernel_18061/1363443833.py:25: SettingWithCopyWarnin

In [16]:
# INDEPENDENT INPUT DATASET - SCALING
independent_gene_expression_data = independent_data[gene_expression_data.columns]
independent_fluxomic_data = independent_data[fluxomic_data.columns]

independent_gene_expression_data_scaled = gene_expression_scaler.transform(
    independent_gene_expression_data
).astype(np.float32)

independent_fluxomic_data_scaled = fluxome_scaler.transform(
    independent_fluxomic_data
).astype(np.float32)

In [18]:
# Load the models
print("Loading model: fluxomics")
flux_model = load_single_view_model_from_file(
    training_data=independent_fluxomic_data, fname="fluxomic"
)
print("Loading model: gene expression")
gene_expression_model = load_single_view_model_from_file(
    training_data=independent_gene_expression_data, fname="gene_expression"
)

print("Loading model: multi-view model (GE+MF)")
# Initialize the multi-view model
multi_model = init_multi_view_model(
    input1_dim=independent_fluxomic_data.shape[1],
    input2_dim=independent_gene_expression_data.shape[1],
    learning_rate=LEARNING_RATE,
    epochs=EPOCHS,
    momentum=MOMENTUM,
    neurons=15,
    model_1=flux_model,
    model_2=gene_expression_model,
)
multi_model.summary()

print("Loading weights: multi_view_model_GE_MF_0.weights.h5")
multi_model.load_weights(DATA_DIR + "multi_view_model_GE_MF_0.weights.h5")

# Predict on the independent dataset
independent_predictions = multi_model.predict(
    [independent_fluxomic_data_scaled, independent_gene_expression_data_scaled]
)
np.savetxt(
    f"{PREDICTIONS_DIR}independent_multi_view_GE_MF_predictions.csv",
    independent_predictions,
    delimiter=",",
)

Loading model: fluxomics
Summary of the single-view model fluxomic


Model: "functional_13"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_8 (InputLayer)      │ (None, 1848)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ fluxomic_1 (Dense)              │ (None, 1000)           │     1,849,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_8 (Dropout)             │ (None, 1000)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ fluxomic_2 (Dense)              │ (None, 1000)           │     1,001,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_9 (Dropout)             │ (None, 1000)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 1)              │         1,001 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,851,001 (10.88 MB)

 Trainable params: 2,851,001 (10.88 MB)

...

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_10      │ (None, 1848)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_11      │ (None, 6170)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ functional_13       │ (None, 1)         │  2,851,001 │ input_layer_10[0… │
│ (Functional)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ functional_15       │ (None, 1)         │  7,173,001 │ input_layer_11[0… │
│ (Functional)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_2       │ (None, 2)         │          0 │ functional_13[0]… │
│ (Concatenate)       │                   │            │ functional_15[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ last_hidden (Dense) │ (None, 15)        │         45 │ concatenate_2[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_8 (Dense)     │ (None, 1)         │         16 │ last_hidden[0][0] │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 10,024,063 (38.24 MB)

 Trainable params: 10,024,063 (38.24 MB)

 Non-trainable params: 0 (0.00 B)

(86, 1848)
(86, 1848)
Loading weights: multi_view_model_GE_MF_0.weights.h5
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
